In [1]:
%run imports.py

/home/ayush/miniconda3/envs/scan/lib/python3.8/site-packages/pymatgen/ext/matproj.py:454: DeprecationWarning:

__init__ is deprecated
MaterialsProjectCompatibility will be updated with new correction classes as well as new values of corrections and uncertainties in 2020



Imports successfully loaded


In [2]:
%run classifiers.py

In [6]:
elem_featurizer = ElementFraction()

In [7]:
binary_formula_ids = loadfn('data/binary_formulas_ids.json')
binary_ids = list(binary_formula_ids.values())

In [9]:
elfcars = loadfn('data/elfcars.json')
ids = [i for i in elfcars.keys()]

In [21]:
skips = ['7356']
vector_list = []
for i in tqdm(binary_ids):
    i = str(i)
    if i in skips:
        new_vec = None
    else:
        new_vec = elem_featurizer.featurize(Composition(elfcars[i].structure.composition.reduced_formula))
    if new_vec is not None:
        vector_list.append(new_vec)

100%|██████████| 427/427 [00:00<00:00, 1718.10it/s]


In [22]:
A = np.vstack(vector_list)
A.shape

(426, 103)

In [55]:
A[0]

array([0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

In [3]:
b = np.load('data/b.npy')
b = np.abs(b) > 0.04

In [23]:
N = A.shape[0]
n = int(0.9*N)
print(N, 'total data points,', n, '(90%) used for model development')
A_train, A_test = A[:n], A[n:]
b_train, b_test = b[:n], b[n:]

426 total data points, 383 (90%) used for model development


In [24]:
classifier = RandomForestClassifier()
classifier.fit(A_train, b_train)
accuracy_score(classifier.predict(A_test), b_test)

0.6744186046511628

In [57]:
sum(b) / len(b)

0.7136150234741784

In [4]:
Arand = np.random.rand(426, 30)
N = Arand.shape[0]
n = int(0.9*N)
print(N, 'total data points,', n, '(90%) used for model development')
A_train, A_test = Arand[:n], Arand[n:]
b_train, b_test = b[:n], b[n:]
classifier = RandomForestClassifier()
classifier.fit(A_train, b_train)
accuracy_score(classifier.predict(A_test), b_test)

426 total data points, 383 (90%) used for model development


0.6511627906976745

In [32]:
bader_statistics = loadfn('data/bader_statistics.json')

In [35]:
element_elfcars = {}
element_formulas_ids = loadfn('data/element_formulas_ids.json')

for f, ID in element_formulas_ids.items():
    element_elfcars[f] = elfcars[str(ID)]
    
element_statistics = {}

for f in element_elfcars.keys():
    spatial_data = element_elfcars[f].get_alpha().data['total'].flatten()
    element_statistics[f] = [np.mean(spatial_data), np.std(spatial_data), np.max(spatial_data)]

In [42]:
def zipper(l1, l2):
    i1, i2 = iter(l1), iter(l2)
    for _ in range(max(len(l1), len(l2))):
        try:
            yield next(i1)
        except StopIteration:
            pass
        try:
            yield next(i2)
        except StopIteration:
            pass
        
zip_lists = lambda l1, l2: [x for x in zipper(l1, l2)]   
flatten_list = lambda l: [y for x in l for y in x]

def trim_vector(vector, length, ndescriptors):
    if len(vector) > length:
        vector = vector[:length]
    elif len(vector) < length:
        idx = 0
        while len(vector) < length:
            vector += vector[idx:idx+ndescriptors]
            idx += ndescriptors
    return vector

skips = ['7356']

def add_general_features(vector, cation, anion, nbader_wells):
    pass

def feature_vector(i, num_wells=2):
    i = str(i)
    if i in skips:
        return None
    cation, anion = [str(e) for e in elfcars[i].structure.composition.elements]
    bader_stats = bader_statistics[i]
    cation_vectors, anion_vectors = [], []
    for well in bader_stats:
        f = [*well.keys()][0]
        vi = np.array(well[f]) - np.array(element_statistics[f])
        if f == cation:
            cation_vectors.append(vi)
        elif f == anion:
            anion_vectors.append(vi)
        else:
            raise ValueError('Atom does not match either cation or anion')
    ndescriptors = len(cation_vectors[0])
    length = num_wells * ndescriptors
    vector = np.array(trim_vector(flatten_list(zip_lists(cation_vectors, anion_vectors)), length, ndescriptors))
#     reshaped = vector.reshape((-1,3))
#     diffsquared_cat_an = np.mean(cation_vectors, axis=0)**2 - np.mean(anion_vectors, axis=0)**2
#     vector = np.append(vector, diffsquared_cat_an)    
    return vector

In [43]:
feature_vector(6405).reshape((-1,3))

array([[ 4.50969334e+00,  4.92817031e-01, -7.41933043e-01],
       [-1.67430955e+02, -3.27421624e+02, -3.03831668e+03]])

In [44]:
vector_list = []
for i in tqdm(binary_ids):
    new_vec = feature_vector(i)
    if new_vec is not None:
        vector_list.append(new_vec)
Asmall = np.vstack(vector_list)
print(Asmall.shape)

100%|██████████| 427/427 [00:00<00:00, 7897.94it/s]

(426, 6)


In [54]:
N = Asmall.shape[0]
n = int(0.9*N)
print(N, 'total data points,', n, '(90%) used for model development')
A_train, A_test = Asmall[:n], Asmall[n:]
b_train, b_test = b[:n], b[n:]
classifier = RandomForestClassifier()
classifier.fit(A_train, b_train)
accuracy_score(classifier.predict(A_test), b_test)

426 total data points, 383 (90%) used for model development


0.6744186046511628